In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from torchvision.utils import make_grid
import torchvision.transforms as T
import sys
from pathlib import Path

from models._styleextraction import StyleExtractor

In [ ]:
def plot_style_embedding(style_stats, use_tsne=True):
    mu = style_stats.mu.squeeze(-1).squeeze(-1).squeeze(-1)  # [L, D, C]
    sig = style_stats.sig.squeeze(-1).squeeze(-1).squeeze(-1)

    L, D, C = mu.shape
    embeddings, labels = [], []

    for layer in range(L):
        for domain in range(D):
            style_vec = torch.cat([mu[layer, domain], sig[layer, domain]], dim=0)
            embeddings.append(style_vec.numpy())
            labels.append(f"L{layer}-D{domain}")

    X = np.stack(embeddings)
    reducer = TSNE(n_components=2, perplexity=5) if use_tsne else PCA(n_components=2)
    X_embedded = reducer.fit_transform(X)

    plt.figure(figsize=(8, 6))
    for i, label in enumerate(labels):
        plt.scatter(X_embedded[i, 0], X_embedded[i, 1], label=label)
    plt.title("t-SNE Embedding: mu + sig")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
def visualize_feature_maps(before, after, num_channels=4):
    """
    Visualises feature maps before and after MixStyle
    before/after: Tensor [B, C, H, W]
    """
    before = before[0, :num_channels]  # first sample
    after = after[0, :num_channels]

    before_grid = make_grid(before.unsqueeze(1), normalize=True, nrow=num_channels).squeeze(0)
    after_grid = make_grid(after.unsqueeze(1), normalize=True, nrow=num_channels).squeeze(0)

    fig, axs = plt.subplots(2, 1, figsize=(12, 5))
    axs[0].imshow(before_grid.cpu(), cmap='viridis')
    axs[0].set_title("Before MixStyle")
    axs[0].axis('off')

    axs[1].imshow(after_grid.cpu(), cmap='viridis')
    axs[1].set_title("After MixStyle")
    axs[1].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
def plot_mu_heatmap(style_stats, channel=0):
    mu = style_stats.mu.squeeze(-1).squeeze(-1).squeeze(-1)  # [L, D, C]
    heatmap_data = mu[:, :, channel].T.numpy()  # [Domain, Layer]
    sns.heatmap(heatmap_data, annot=True, fmt=".2f", cmap="magma")
    plt.xlabel("Layer")
    plt.ylabel("Domain")
    plt.title(f"mu Heatmap (Channel {channel})")
    plt.show()

In [ ]:
def plot_std_heatmap(style_stats, channel=0):
    sig = style_stats.sig.squeeze(-1).squeeze(-1).squeeze(-1)  # [L, D, C]
    heatmap_data = sig[:, :, channel].T.numpy()  # [Domain, Layer]
    sns.heatmap(heatmap_data, annot=True, fmt=".2f", cmap="magma")
    plt.xlabel("Layer")
    plt.ylabel("Domain")
    plt.title(f"std Heatmap (Channel {channel})")
    plt.show()

In [ ]:
style = StyleExtractor
style_stats = style.load("style_stats.json")

# t-SNE Embedding
plot_style_embedding(style_stats)

# feature map comparison (f.e. from one training run)
# before = some_feature_map.clone()
# after = mixstyle_layer(some_feature_map)
# visualize_feature_maps(before, after)

# heatmap
plot_mu_heatmap(style_stats, channel=0)
plot_std_heatmap(style_stats, channel=0)


In [3]:
import sys
from pathlib import Path

# 1. Korrekten Projektroot setzen
project_root = Path("/home/hahlers/mixStyle-TTA")  # ODER: Path.cwd().parent
print(f"Projektroot: {project_root}")

# 2. Pfadüberprüfung
tuning_path = project_root / "models" / "tuning"
hptuning_file = tuning_path / "_hyperparameter_tuning.py"
print(f"Existiert tuning Modul?: {hptuning_file.exists()}")

# 3. Pfad zum Systempfad hinzufügen
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 4. Import mit Debugging
try:
    from models.tuning._hyperparameter_tuning import HP_Tuner
    print("Import erfolgreich!")
    
    # Testausführung
    test_path = project_root / "experiments" / "hp_results"
    if test_path.exists():
        result = HP_Tuner.compute_global_best_params(str(test_path))
        print(f"Beste Genauigkeit: {result['mean_val_acc']:.2%}")
    else:
        print(f"Testpfad {test_path} existiert nicht!")
except ImportError as e:
    print(f"Import fehlgeschlagen: {e}")
    print("Mögliche Lösungen:")
    print("1. Sicherstellen, dass '_hyperparameter_tuning.py' existiert")
    print("2. __init__.py Dateien in models/ und models/tuning/ erstellen")
    print("3. Absoluten Pfad verwenden:")
    print(f"   from {tuning_path}._hyperparameter_tuning import HP_Tuner")

Projektroot: /home/hahlers/mixStyle-TTA
Existiert tuning Modul?: True
Import erfolgreich!


IndexError: single positional indexer is out-of-bounds

In [1]:
import torch
import torchvision.models as models

# Load a pre-trained ResNet model
model = models.resnet50(pretrained=True)

# Print the model's architecture
print(model)

/home/hahlers/miniconda3/envs/server_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hahlers/miniconda3/envs/server_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/hahlers/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 105MB/s] 


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from models._resnet import resnet50

model = resnet50(pretrained=True, num_classes=7, num_domains=4)
print(model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchN